In [ ]:
# 1. 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.preprocessing import LabelEncoder

# 한글 폰트 설정
import matplotlib.font_manager as fm
try:
    fe = fm.FontEntry(
        fname=r'../../font/NanumFont/NanumGothic.ttf',
        name='NanumGothic')
    fm.fontManager.ttflist.insert(0, fe)
    plt.rcParams.update({'font.size': 12, 'font.family': 'NanumGothic'})
    plt.rc('font', family='NanumGothic')
except FileNotFoundError:
    print("나눔고딕 폰트를 찾을 수 없어 기본 폰트로 설정됩니다.")
    pass

# --- 데이터 로딩 및 전처리 (모델 훈련 시와 동일한 과정) ---
# 저장된 모델에 맞는 입력 데이터를 만들기 위해 필수적인 과정입니다.

try:
    print(">> 데이터 로딩 및 전처리를 시작합니다...")
    # 2. 경로 설정
    MODEL_DIR = '../../model'
    MODEL_FILENAME = 'final_price_model.pkl'
    MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME)

    # 3. 데이터 로드
    train_df_clean = pd.read_csv('../../data/processed/cleaned_data/train_clean.csv')
    test_df_clean = pd.read_csv('../../data/processed/cleaned_data/test_clean.csv')
    
    # isTest 컬럼으로 구분 후 병합
    train_df_clean['isTest'] = 0
    test_df_clean['isTest'] = 1
    df = pd.concat([train_df_clean, test_df_clean], ignore_index=True)

    # 4. 피처 엔지니어링
    # 위치 기반 통계 특성
    sigungu_stats = df.groupby('자치구')['target'].agg(['mean','std','count']).reset_index()
    sigungu_stats.columns = ['자치구','자치구별_평균가격','자치구별_가격편차','자치구별_거래량']
    df = pd.merge(df, sigungu_stats, on='자치구', how='left')

    df['자치구별_가격편차'].fillna(0, inplace=True)
    df['자치구별_평균가격'].fillna(df['자치구별_평균가격'].mean(), inplace=True)
    df['자치구별_거래량'].fillna(df['자치구별_거래량'].mean(), inplace=True)

    # 상호작용 특성
    df['면적_x_연식'] = df['전용면적'] * df['연식']
    df['면적_x_층'] = df['전용면적'] * df['층']
    
    # 5. 데이터 분리 및 정제
    train_df = df[df['isTest']==0].drop('isTest',axis=1).copy()
    test_df  = df[df['isTest']==1].drop('isTest',axis=1).copy()
    
    train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    train_df.dropna(inplace=True)
    
    test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    test_df.fillna(0, inplace=True)
    
    features = [col for col in train_df.columns if col not in ['target', 'id', '아파트이름']]
    X_train_raw = train_df[features]
    y_train     = train_df['target']
    X_test_raw  = test_df[features]

    # 6. 범주형 피처 라벨 인코딩
    categorical_features = X_train_raw.select_dtypes(include=['object']).columns.tolist()
    for col in categorical_features:
        le = LabelEncoder()
        all_vals = pd.concat([X_train_raw[col], X_test_raw[col]]).astype(str).unique()
        le.fit(all_vals)
        X_train_raw[col] = le.transform(X_train_raw[col].astype(str))
        X_test_raw[col]  = le.transform(X_test_raw[col].astype(str))
    
    print(">> 데이터 준비 완료.")

    # --- 모델 로드 및 예측 ---
    print(f">> 저장된 모델 '{MODEL_FILENAME}'을(를) 불러옵니다...")
    loaded_model = joblib.load(MODEL_PATH)
    print(">> 모델 로드 완료.")

    # 학습 및 테스트 데이터에 대한 예측 생성
    train_preds = loaded_model.predict(X_train_raw)
    test_preds = loaded_model.predict(X_test_raw)
    print(">> 예측 완료.")


    # --- 결과 시각화 ---
    print("\n>> 모델 결과 시각화를 시작합니다...")
    
    # 1. 피처 중요도(Feature Importance) 시각화
    feature_importances = pd.DataFrame(
        {'feature': X_train_raw.columns, 'importance': loaded_model.feature_importances_}
    ).sort_values('importance', ascending=False)

    plt.figure(figsize=(12, 10))
    sns.barplot(x='importance', y='feature', data=feature_importances.head(20))
    plt.title('상위 20개 피처 중요도 (불러온 모델)', fontsize=16)
    plt.xlabel('중요도', fontsize=12)
    plt.ylabel('피처', fontsize=12)
    plt.grid(True, axis='x', linestyle='--')
    plt.tight_layout()
    plt.show()

    # 2. 실제 값 vs 학습 데이터 예측 값 비교 (Scatter Plot)
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x=y_train, y=train_preds, alpha=0.3)
    plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=2, label='이상적인 예측선 (y=x)')
    plt.title('실제 값 vs 학습 데이터 예측 값 비교', fontsize=16)
    plt.xlabel('실제 값 (Actual)', fontsize=12)
    plt.ylabel('학습 데이터 예측 값 (Predicted)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 3. 잔차(Residuals) 분포 확인 (Histogram)
    residuals = y_train - train_preds
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, bins=50)
    plt.title('잔차(실제-예측) 분포', fontsize=16)
    plt.xlabel('잔차 (Residuals)', fontsize=12)
    plt.ylabel('빈도 (Frequency)', fontsize=12)
    plt.axvline(x=0, color='red', linestyle='--')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 4. 학습 데이터와 테스트 예측 결과 분포 비교 (KDE Plot)
    plt.figure(figsize=(10, 6))
    sns.kdeplot(y_train, label='학습 데이터 실제 값', color='blue', fill=True)
    sns.kdeplot(test_preds, label='테스트 데이터 예측 값', color='orange', fill=True)
    plt.title('학습 데이터와 테스트 예측의 분포 비교', fontsize=16)
    plt.xlabel('아파트 가격', fontsize=12)
    plt.ylabel('밀도 (Density)', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

except FileNotFoundError as e:
    print(f"[오류] 파일 로드 실패: {e}")
    print("데이터 파일 경로('../../data/processed/cleaned_data/')나 모델 파일 경로('../../model/')를 확인해주세요.")
except Exception as e:
    print(f"[오류] 시각화 중 예기치 않은 오류 발생: {e}")

